In [1]:
# import urllib.request
# api_key = ''
# crypto = 'DOGE'
# interval = '1d'
# # url = "https://api.nomics.com/v1/currencies/ticker?key=" + api_key + \
# #       "&ids=" + crypto + "&interval=" + interval + "&convert=USD&per-page=100&page=1"

# start_date = '2020-01-01'
# end_date = '2020-12-31'
# url = "https://api.nomics.com/v1/currencies/sparkline?key=" + api_key + \
#       "&ids=" + crypto + "&start=" + start_date + "T00%3A00%3A00Z&end=" + end_date + "T00%3A00%3A00Z" + "&interval=" + interval

# print(urllib.request.urlopen(url).read())

In [12]:
# Import the libraries
import numpy as np
import pandas as pd

In [27]:
# Load the data
df = pd.read_csv('doge.csv')

# Set date as the index 
df.set_index(pd.DatetimeIndex(df['date']), inplace=True)

# Remove unwanted data
df = df[['PriceUSD']]
df = df.iloc[2200:,:]

# Show the data
df.head()

,PriceUSD
date,
2020-02-01,0.002390
2020-02-02,0.002469
2020-02-03,0.002413
2020-02-04,0.002492
2020-02-05,0.002564


In [28]:
# Create a variable to store the number of days into the future that we want to predict
prediction_days = 1

# Create a new column called Prediction
df['Prediction'] = df['PriceUSD'].shift(-prediction_days)

# Show the data
df

,PriceUSD,Prediction
date,,
2020-02-01,0.002390,0.002469
2020-02-02,0.002469,0.002413
2020-02-03,0.002413,0.002492
2020-02-04,0.002492,0.002564
2020-02-05,0.002564,0.002663
...,...,...
2021-06-22,0.188829,0.234789
2021-06-23,0.234789,0.263518
2021-06-24,0.263518,0.240247


In [33]:
# Create the independent dataset (X)
X = np.array(df.drop(['Prediction'], 1))

# Remove the last n+1 rows of data where n is the prediction_days
X = X[: len(df) - prediction_days - 1]

# Show the data
X[-10:]

array([[0.3206744 ],
       [0.30715372],
       [0.30584986],
       [0.29246527],
       [0.28729606],
       [0.28066914],
       [0.17918748],
       [0.18882902],
       [0.23478921],
       [0.26351766]])

In [32]:
# Create a dependent dataset(y)
y = np.array(df['Prediction'])

# Get all of the y values except the last n+1 rows
y = y[: -prediction_days-1]

# Show the data
y[-10:]

array([0.30715372, 0.30584986, 0.29246527, 0.28729606, 0.28066914,
       0.17918748, 0.18882902, 0.23478921, 0.26351766, 0.24024652])

In [2]:
# !pip install sklearn

In [36]:
# Split the data into train and test dataset
from sklearn.model_selection import train_test_split

x_train, x_test, y_train, y_test = train_test_split(X, y, test_size = 0.2)  

In [39]:
from sklearn.ensemble import RandomForestRegressor

forest = RandomForestRegressor(n_estimators = 2, random_state = 587)
forest.fit(x_train, y_train)
print(forest.score(x_test, y_test))

0.9496067955111462


In [40]:
# Show how close the predicted values and the actual values are
predicted = forest.predict(x_test)

print(predicted)
print()

print(y_test)

[0.00197063 0.00261834 0.00181496 0.00251305 0.00275109 0.00261834
 0.00232458 0.00256445 0.00272815 0.00311065 0.31929102 0.05144286
 0.00242425 0.22617334 0.25905308 0.00284642 0.00453796 0.40528833
 0.00261834 0.00185949 0.00264089 0.40528833 0.00210012 0.00344347
 0.01030772 0.0032178  0.0034584  0.1220027  0.0027274  0.05405095
 0.00344    0.0504182  0.00263743 0.0019295  0.00245752 0.00360488
 0.05663639 0.00260422 0.00332727 0.00263317 0.00249171 0.00315519
 0.00384714 0.31929102 0.05403362 0.00252266 0.22617334 0.31186969
 0.09307335 0.00273411 0.05742943 0.00186709 0.00343888 0.00308966
 0.00973015 0.00262512 0.00265579 0.00265785 0.00256602 0.05484354
 0.00261597 0.29038787 0.05599572 0.00162372 0.00323671 0.00339019
 0.2839826  0.00353832 0.05144286 0.00254801 0.00277354 0.00262452
 0.00870081 0.49579423 0.0034584  0.00248037 0.00292296 0.00210012
 0.00184372 0.00322578 0.00343888 0.49101729 0.00256161 0.05424381
 0.00339019 0.00232458 0.40528833 0.0592477  0.00172901 0.0032

In [42]:
# Get the validation data of the model
# Create a variable to store all of the rows in the data set except the last n rows
temp_df = df[: -prediction_days]

# Create a variable to store the independent price value
x_value = temp_df.tail(1)['PriceUSD'][0]

print(x_value)

0.240246524


In [47]:
prediction = forest.predict([[x_value]])

print('The price of Dogecoin in', prediction_days, 'day(s) is predicted to be', prediction[0])

print('The actual price was', temp_df.tail(1)['Prediction'][0])

The price of Dogecoin in 1 day(s) is predicted to be 0.26351766
The actual price was 0.24094331
